# **Preparación de datos previa a proceso de modelado**

## **Librerías**

In [1]:
from modeling_auxiliary_functions import (add_sin_cos_transforms, 
                                 )
import warnings
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
warnings.filterwarnings('ignore')

## **Transformaciones**

###
**Lectura y unión de datasets**

In [3]:
df = pd.read_pickle('dataset.pkl')
print("Shape:", df.shape)
df.head(3)

Shape: (35060, 82)


,time,temp_ Barcelona,temp_Bilbao,temp_Madrid,temp_Seville,temp_Valencia,temp_min_ Barcelona,temp_min_Bilbao,temp_min_Madrid,temp_min_Seville,temp_min_Valencia,temp_max_ Barcelona,temp_max_Bilbao,temp_max_Madrid,temp_max_Seville,temp_max_Valencia,pressure_ Barcelona,pressure_Bilbao,pressure_Madrid,pressure_Seville,pressure_Valencia,humidity_ Barcelona,humidity_Bilbao,humidity_Madrid,humidity_Seville,humidity_Valencia,wind_speed_ Barcelona,wind_speed_Bilbao,wind_speed_Madrid,wind_speed_Seville,wind_speed_Valencia,wind_deg_ Barcelona,wind_deg_Bilbao,wind_deg_Madrid,wind_deg_Seville,wind_deg_Valencia,rain_1h_ Barcelona,rain_1h_Bilbao,rain_1h_Madrid,rain_1h_Seville,rain_1h_Valencia,rain_3h_ Barcelona,rain_3h_Bilbao,rain_3h_Madrid,rain_3h_Seville,rain_3h_Valencia,snow_3h_ Barcelona,snow_3h_Bilbao,snow_3h_Madrid,snow_3h_Seville,snow_3h_Valencia,clouds_all_ Barcelona,clouds_all_Bilbao,clouds_all_Madrid,clouds_all_Seville,clouds_all_Valencia,weather_main_ Barcelona,weather_main_Bilbao,weather_main_Madrid,weather_main_Seville,weather_main_Valencia,weather_description_ Barcelona,weather_description_Bilbao,weather_description_Madrid,weather_description_Seville,weather_description_Valencia,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,total load forecast,total load actual
0,2015-01-01 00:00:00,281.625,269.657312,267.325000,273.375,270.475,281.625,269.657312,267.325,273.375,270.475,281.625,269.657312,267.325000,273.375,270.475,1035.000000,1070.205106,1070.205106,1070.205106,1001.0,100.0,97.0,63.0,75.0,77.0,7.0,0.0,1.0,1.0,1.0,58.0,226.0,309.0,21.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,clear,clear,clear,clear,clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,447.0,329.0,4844.0,4821.0,298.324249,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,26118.0,25385.0
1,2015-01-01 01:00:00,281.625,269.763500,267.325000,273.375,270.475,281.625,269.763500,267.325,273.375,270.475,281.625,269.763500,267.325000,273.375,270.475,1035.000000,1035.000000,1070.205106,1070.205106,1001.0,100.0,97.0,63.0,75.0,77.0,7.0,0.0,1.0,1.0,1.0,58.0,229.0,309.0,21.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,clear,clear,clear,clear,clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,449.0,328.0,5196.0,4755.0,298.324249,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,24934.0,24382.0
2,2015-01-01 02:00:00,281.286,269.251688,289.708181,274.086,269.686,281.286,269.251688,266.186,274.086,269.686,281.286,269.251688,291.172867,274.086,269.686,1070.205106,1070.205106,1070.205106,1070.205106,1002.0,100.0,97.0,64.0,71.0,78.0,7.0,1.0,1.0,3.0,0.0,48.0,224.0,273.0,27.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,clear,clear,clear,clear,clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,448.0,323.0,4857.0,4581.0,298.324249,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,23515.0,22734.0


###
**Transformaciones temporales**

In [4]:
df = add_sin_cos_transforms(df, ["month", "dayofweek", "hour"])
df.head(3)

,time,temp_ Barcelona,temp_Bilbao,temp_Madrid,temp_Seville,temp_Valencia,temp_min_ Barcelona,temp_min_Bilbao,temp_min_Madrid,temp_min_Seville,temp_min_Valencia,temp_max_ Barcelona,temp_max_Bilbao,temp_max_Madrid,temp_max_Seville,temp_max_Valencia,pressure_ Barcelona,pressure_Bilbao,pressure_Madrid,pressure_Seville,pressure_Valencia,humidity_ Barcelona,humidity_Bilbao,humidity_Madrid,humidity_Seville,humidity_Valencia,wind_speed_ Barcelona,wind_speed_Bilbao,wind_speed_Madrid,wind_speed_Seville,wind_speed_Valencia,wind_deg_ Barcelona,wind_deg_Bilbao,wind_deg_Madrid,wind_deg_Seville,wind_deg_Valencia,rain_1h_ Barcelona,rain_1h_Bilbao,rain_1h_Madrid,rain_1h_Seville,rain_1h_Valencia,rain_3h_ Barcelona,rain_3h_Bilbao,rain_3h_Madrid,rain_3h_Seville,rain_3h_Valencia,snow_3h_ Barcelona,snow_3h_Bilbao,snow_3h_Madrid,snow_3h_Seville,snow_3h_Valencia,clouds_all_ Barcelona,clouds_all_Bilbao,clouds_all_Madrid,clouds_all_Seville,clouds_all_Valencia,weather_main_ Barcelona,weather_main_Bilbao,weather_main_Madrid,weather_main_Seville,weather_main_Valencia,weather_description_ Barcelona,weather_description_Bilbao,weather_description_Madrid,weather_description_Seville,weather_description_Valencia,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,total load forecast,total load actual,sin_month,cos_month,sin_dayofweek,cos_dayofweek,sin_hour,cos_hour
0,2015-01-01 00:00:00,281.625,269.657312,267.325000,273.375,270.475,281.625,269.657312,267.325,273.375,270.475,281.625,269.657312,267.325000,273.375,270.475,1035.000000,1070.205106,1070.205106,1070.205106,1001.0,100.0,97.0,63.0,75.0,77.0,7.0,0.0,1.0,1.0,1.0,58.0,226.0,309.0,21.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,clear,clear,clear,clear,clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,447.0,329.0,4844.0,4821.0,298.324249,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,26118.0,25385.0,0.5,0.866025,1.224647e-16,-1.0,0.000000,1.000000
1,2015-01-01 01:00:00,281.625,269.763500,267.325000,273.375,270.475,281.625,269.763500,267.325,273.375,270.475,281.625,269.763500,267.325000,273.375,270.475,1035.000000,1035.000000,1070.205106,1070.205106,1001.0,100.0,97.0,63.0,75.0,77.0,7.0,0.0,1.0,1.0,1.0,58.0,229.0,309.0,21.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,clear,clear,clear,clear,clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,449.0,328.0,5196.0,4755.0,298.324249,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,24934.0,24382.0,0.5,0.866025,1.224647e-16,-1.0,0.269797,0.962917
2,2015-01-01 02:00:00,281.286,269.251688,289.708181,274.086,269.686,281.286,269.251688,266.186,274.086,269.686,281.286,269.251688,291.172867,274.086,269.686,1070.205106,1070.205106,1070.205106,1070.205106,1002.0,100.0,97.0,64.0,71.0,78.0,7.0,1.0,1.0,3.0,0.0,48.0,224.0,273.0,27.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,clear,clear,clear,clear,clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,448.0,323.0,4857.0,4581.0,298.324249,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,23515.0,22734.0,0.5,0.866025,1.224647e-16,-1.0,0.519584,0.854419


###
**Transformaciones de representación de perídos pasados**

In [5]:
# la media de la columna X de las ultimas 4 semanas en la hora H como variable predictora de lapso del futuro.
# para hacerlo con clima primero hay que hacer pivot de las ciudades.

## **Modelado predictivo**

El objetivo es predecir las 24 horas de la demanda total de energía con una anticipación de 12,24 y 36 horas según las sesiones intradiarias del mercado eléctrico mayorista.

In [6]:
df

,time,temp_ Barcelona,temp_Bilbao,temp_Madrid,temp_Seville,temp_Valencia,temp_min_ Barcelona,temp_min_Bilbao,temp_min_Madrid,temp_min_Seville,temp_min_Valencia,temp_max_ Barcelona,temp_max_Bilbao,temp_max_Madrid,temp_max_Seville,temp_max_Valencia,pressure_ Barcelona,pressure_Bilbao,pressure_Madrid,pressure_Seville,pressure_Valencia,humidity_ Barcelona,humidity_Bilbao,humidity_Madrid,humidity_Seville,humidity_Valencia,wind_speed_ Barcelona,wind_speed_Bilbao,wind_speed_Madrid,wind_speed_Seville,wind_speed_Valencia,wind_deg_ Barcelona,wind_deg_Bilbao,wind_deg_Madrid,wind_deg_Seville,wind_deg_Valencia,rain_1h_ Barcelona,rain_1h_Bilbao,rain_1h_Madrid,rain_1h_Seville,rain_1h_Valencia,rain_3h_ Barcelona,rain_3h_Bilbao,rain_3h_Madrid,rain_3h_Seville,rain_3h_Valencia,snow_3h_ Barcelona,snow_3h_Bilbao,snow_3h_Madrid,snow_3h_Seville,snow_3h_Valencia,clouds_all_ Barcelona,clouds_all_Bilbao,clouds_all_Madrid,clouds_all_Seville,clouds_all_Valencia,weather_main_ Barcelona,weather_main_Bilbao,weather_main_Madrid,weather_main_Seville,weather_main_Valencia,weather_description_ Barcelona,weather_description_Bilbao,weather_description_Madrid,weather_description_Seville,weather_description_Valencia,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,total load forecast,total load actual,sin_month,cos_month,sin_dayofweek,cos_dayofweek,sin_hour,cos_hour
0,2015-01-01 00:00:00,281.625,269.657312,267.325000,273.375,270.475,281.625,269.657312,267.325,273.375,270.475,281.625,269.657312,267.325000,273.375,270.475,1035.000000,1070.205106,1070.205106,1070.205106,1001.0,100.0,97.0,63.0,75.0,77.0,7.0,0.0,1.0,1.0,1.0,58.0,226.0,309.0,21.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,clear,clear,clear,clear,clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,447.0,329.0,4844.0,4821.0,298.324249,863.000000,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,26118.0,25385.0,5.000000e-01,0.866025,1.224647e-16,-1.0,0.000000e+00,1.000000
1,2015-01-01 01:00:00,281.625,269.763500,267.325000,273.375,270.475,281.625,269.763500,267.325,273.375,270.475,281.625,269.763500,267.325000,273.375,270.475,1035.000000,1035.000000,1070.205106,1070.205106,1001.0,100.0,97.0,63.0,75.0,77.0,7.0,0.0,1.0,1.0,1.0,58.0,229.0,309.0,21.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,clear,clear,clear,clear,clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,449.0,328.0,5196.0,4755.0,298.324249,920.000000,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,24934.0,24382.0,5.000000e-01,0.866025,1.224647e-16,-1.0,2.697968e-01,0.962917
2,2015-01-01 02:00:00,281.286,269.251688,289.708181,274.086,269.686,281.286,269.251688,266.186,274.086,269.686,281.286,269.251688,291.172867,274.086,269.686,1070.205106,1070.205106,1070.205106,1070.205106,1002.0,100.0,97.0,64.0,71.0,78.0,7.0,1.0,1.0,3.0,0.0,48.0,224.0,273.0,27.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,clear,clear,clear,clear,clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,448.0,323.0,4857.0,4581.0,298.324249,1164.000000,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,23515.0,22734.0,5.000000e-01,0.866025,1.224647e-16,-1.0,5.195840e-01,0.854419
3,2015-01-01 03:00:00,281.286,269.203344,289.708181,274.086,269.686,281.286,269.203344,266.186,274.086,269.686,281.286,269.203344,291.172867,274.086,269.686,1070.205106,1035.000000,1070.205106,1070.205106,1002.0,100.0,97.0,64.0,71.0,78.0,7.0,1.0,1.0,3.0,0.0,48.0,225.0,273.0,27.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,clear,clear,clear,clear,clear,sky is clear,sky is clear,sky 